In [35]:
import numpy as np
import pandas as pd
import json
import tweepy
import seaborn as sns
import os
import requests as req
import calendar
import pytz
import time
import got3

from datetime import datetime, date, timedelta

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()



In [36]:
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('keys')))
file_name = os.path.join(api_dir + "//keys", "api_keys.json")
data = json.load(open(file_name))

gkey = data['google_api_key']
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']


In [37]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())



In [38]:
#coin list by symbol in descending order of market cap
#bitcoin - ethereum - ripple - bitcoin cash - cardano - litecoin - stella - neo - eos - xem  
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'ADA', 'LTC', 'XLM', 'NEO', 'EOS', 'XEM']



In [39]:
#URLs
social_stats_url = "https://www.cryptocompare.com/api/data/socialstats/?id="

coin_list_url = "https://www.cryptocompare.com/api/data/coinlist/"

coin_historical_url = "https://min-api.cryptocompare.com/data/pricehistorical?fsym="
coin_historical_url_second = "&tsyms=USD&ts="



In [40]:
#variables 
coin_list_response = []

coin_data_df = pd.DataFrame(columns=["id", "symbol", "coin_name", "coin_price", "date", 
                                     "compound_average",
                                     "positive_average", "neutral_average", "negative_average"
                                   ])


coin_data_df.set_index("symbol")
coin_data_df.head()

,id,symbol,coin_name,coin_price,date,compound_average,positive_average,neutral_average,negative_average


In [41]:
#date
year = 2018
month = 1
day = 1

#sampling max number of tweets per day per coin
max_tweets = 25

In [42]:
def GetCoinList():
    response = req.get(coin_list_url).json()

    count = 0
    for each in response["Data"]:
        count = count + 1
    
    print(count)
    return response


def GetCoinId(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return row["Id"]


def GetCoinName(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            name = row["CoinName"].lower()
            if 

               
def GetHistoricalPrice(symbol, utctime):
    #url to get historical price
    url = coin_historical_url + symbol + coin_historical_url_second + str(utctime)
    #get the historical price
    price = req.get(url).json()
    return price[symbol]["USD"]


def GetSocialStats(coin_id):
    url = social_stats_url + coin_id 
    return req.get(url).json()


def GetUtcTime(year,month,day):
    utc = calendar.timegm(datetime(year,month,day).utctimetuple())
    #move to the next day
    next_date = date(year,month,day) + timedelta(1)
    year = next_date.year
    month = next_date.month
    day = next_date.day
    return utc
    
def SetValues(symbol,row):
    coin_data_df.set_value(symbol, "id", int(row["id"]))
    coin_data_df.set_value(symbol, "symbol", row["symbol"])
    
    new_date = date(year,month,day).strftime("%Y-%m-%d")  
    coin_data_df.set_value(symbol, "date", new_date)

    coin_data_df.set_value(symbol, "coin_name", row["coin_name"])
    coin_data_df.set_value(symbol, "coin_price", float(row["coin_price"]))
    coin_data_df.set_value(symbol, "compound_average", float(row["compound_average"]))
    coin_data_df.set_value(symbol, "positive_average", float(row["positive_average"]))
    coin_data_df.set_value(symbol, "neutral_average", float(row["neutral_average"]))
    coin_data_df.set_value(symbol, "negative_average", float(row["negative_average"]))
    

    
def AnalyzeTweets(coin_name, until_date):
   
    tweetCriteria = got3.manager.TweetCriteria().setUntil(until_date).setQuerySearch(coin_name).setMaxTweets(max_tweets)

    for i in range(max_tweets):
        #each tweet
        tweet = got3.manager.TweetManager.getTweets(tweetCriteria)[i]      
        #run Vader analysis
        compound = analyzer.polarity_scores(tweet.text)["compound"]
        pos = analyzer.polarity_scores(tweet.text)["pos"]
        neu = analyzer.polarity_scores(tweet.text)["neu"]
        neg = analyzer.polarity_scores(tweet.text)["neg"]
        #add each score to an appropriate array
        compound_list.append(compound)
        positive_list.append(pos)
        negative_list.append(neg)
        neutral_list.append(neu)





In [43]:
#get coin list
coin_list_response = GetCoinList()

count = 0

#build coin data
while count < 2:
    #get UTC time, one day at a time
    utc_time = calendar.timegm(datetime(year,month,day).utctimetuple())
    until_date = date(year,month,day).strftime("%Y-%m-%d")  
 
    #process one coin at a time
    for symbol in coin_list:
        coin_name = GetCoinName(symbol)
        print(coin_name)
        
        #calculate Vader scores
        compound_list = []
        positive_list = []
        negative_list = []
        neutral_list = []
        AnalyzeTweets(coin_name,until_date)
        
        print(np.mean(compound_list))
        
        row = {"id": GetCoinId(symbol), 
               "symbol": symbol, 
               "date": utc_time,
               "coin_name": coin_name,
               "coin_price": GetHistoricalPrice(symbol,utc_time),
               "compound_average": np.mean(compound_list),
               "positive_average": np.mean(positive_list), 
               "neutral_average": np.mean(neutral_list), 
               "negative_average": np.mean(negative_list) }

        SetValues(symbol,row)

    count = count + 1
    

    
coin_data_df.head()

2111
bitcoin
Vitalik come back to bitcoin! (BCH)
2017 Review: The Year Cryptocurrencies Seized the Mainstream Spotlight: http://ift.tt/2q85cbE # bitcoin # btc
https://www.newexpressnews.com/which-cryptocurrency-will-be-the-next-bitcoin/ … @ Newexpressnews A dominant style for the year ahead will likely be the hunt for the next bitcoin, and Ripple appears to be making the very best claim for that mantle today. pic.twitter.com/nWGPSlEIc6
"Bitcoin balon değil, iğne"
Happy New Year to all # Bitcoin and Crypto Enthusiasts. This is year of Bitcoin and Crypto
The Greatest Bitcoin Memes of 2017 https://news.bitcoin.com/greatest-bitcoin-memes-2017/ … via @ BTCTN
If you were a banker and wanted to kill bitcoin and ETH, the only way to do it is by joining, making people trade for their version of it. And they have to actually bring investors to the table with a great product to do that. Just look at the media hype they brought in. Good+Bad
I transferred litecoin from Coinbase to Kraken. Well sinc

Yo @ 6BillionPeople ! you in on @ ElectracoinECA??? Get in before its too late bro. Im 20X right now! # CryptoCurrencies # bitcoin # Litecoin # ETHEREUM # Ripple
Ethereum Foundation Releases Alpha Casper Proof of Stake T http://bit.ly/2DDSxPp (http://bit.ly/2DDPEhu ) via @ sethbannon
0.23534
ripple
Less thaN a MinUge
https://www.newexpressnews.com/which-cryptocurrency-will-be-the-next-bitcoin/ … @ Newexpressnews A dominant style for the year ahead will likely be the hunt for the next bitcoin, and Ripple appears to be making the very best claim for that mantle today. pic.twitter.com/nWGPSlEIc6
Predictions of 2018: 1. iPhone Xs/Xi will have a fingerprint sensor embedded in the screen. So will Samsung's S9. 2. Warriors will win 3rd Championship in 4 years. 3. 8k TVs will go into market. 4. Total wireless and contactless chargers will be in use. 5. Ripple will hit $100
pic.twitter.com/xBjCf0PKhn
Umm, Riverstar? I’m not sure that is a good idea, Hawkwing could barely handle a couple and he’

Bitcoin Cash: $2,440.20 +16.39% (+$343.64) High: $2,470 Low: $2,080.00 Volume: 3206 $ BCC # BCC # bitcoincash
Happy New Year Santa wish you a wonderful holiday season! $ XVG # Bitcoin # BTC $ BTC # Litecoin # LTC $ LTC # ETH $ ETH # Ripple # XRP # Electroneum # ETN # BitcoinCash # BCH # BCC $ BCH $ BCC # alcoin # litecoin # BTC # NEo # IOTA # Ripple # crypto # Altcoins # ethconnect # IOTA # Qtum # BITSpic.twitter.com/F2EaN8xCuF
0.08062
cardano
Everyone that holds Verge, Cardano or TRON look at your accounts today. Look at your accounts today!
Best # NYE # party going to be looking at # coinmarketcap with 2018 making the # cryptospace go to the moon even further. # altcoins # bitcoin # litecoin # ethos # salt # SNT # cardano # iota # Quantstamp # RequestNetwork # ETHEREUM # Verge # ripple # newaltcoins $ pink $ opus $ ufr # HODLpic.twitter.com/PFDB6yvDCA
What's your favorite # coin for 2018? # Ripple? # VergeCoin? # Cardano? tough to pick one! Tomorrow we'll show you one of the most pro

augto, LTC LITECOIN EXCHANGE - NO ID VERIFICATION ➜➜ …http://binance.cryptocurrency.trading.gewgool.com/b/glosmwezf523945007 … XRP ♳ IOTA ♰ LTC ♬ BCC ☻ XVG BUY ☆ ETH 523945007kgidjzqyc tyuogjzfd
0.055248
stellar
Hi guyz. What is your prediction for # stellar price in 2018 ? # cryptocurrency # CryptoMonnaie # StellarLumens
Wow! What a stellar list of events over the past year that focused on improvement in the quality of maternal... https://fb.me/8oAYcYd6u
I'm getting my top 10 TV episodes list together and I've limited myself to no more than 2 episodes of any one show and gah. IT'S SO HARD! Both HACF and Black Sails had STELLAR final seasons.
11. Love Cherry Motion - Choerry This song was originally made for Jinsoul and she was gonna Stellar-vibrato-grind in the floor on every drop! pic.twitter.com/i96F2R1WlS
TFW you destroy fact-denying Trumpites about President Obama's stellar jobs record. pic.twitter.com/WIefTFJbIR
The writing is so stellar. My best friend @ JenniferBrody (who is an

あれもでしたか…
_(:3 」∠)_フラワーみたいなー
サービス良いですね。
0.094384
nem
queria muito e olha que eu nem sou de beber https://twitter.com/markblackfaerie/status/947617116127154178 …
Não postei não, nem tirei nenhuma kkk
Sorte q ngm reprovo se n ia da uma treta isso bixooo “essa é minha caneta? Nunca nem vi”
Nem vou botar meu celular no silencioso pq sei q ngm vai ligar
não estou nem um pouco preparada para rever minha prima pic.twitter.com/4Sm2QgC4gf
2018, nem te conheço e te considero FODA
49ers 34 x 6 Rams ainda bem q eu nem preciso mais do meu gurley
Vou começar 2018 que nem foi o desse ano... só tomando no cu.
eu ODEIO gente insensível da porra querendo ganhar atenção por causa da morte dele, isso não é homenagem nem fodendo
minha mae é muito linda ela parece uma adolescente de tão linda e arrumadinha que ela é, nem parece ser mãe assim
A bruza que to usando nem precisa por sutiã, ai que tudooooo
"nem bebo pô"
No fim eu nem pesquei
Tu sentes quando alguém já não está nem aí pra ti .
minha bad bateu bem 

Top 5 ETH Trading Pairs 24hrs Volume 1. ETH/BTC: 891,531 ETH 2. ETH/USD: 357,394 ETH 3. ETH/USDT: 135,204 ETH 4. ETH/KRW: 99,305 ETH 5. ETH/EUR: 49,575 ETH Click Here http://goo.gl/6X8xNx To Join The Trading Fun @Binance via @ bllshbrsh # Ethereum
Want to know more about the @ BudboApp's Blockchain solution for Global Cannabis Industry? Visit https://budbo.io/?utm_source=BTT ! The pre-sale is going on!!! # BudboICO # Blockchain # ICO # Ethereum # BTC # Budbo
Crypto Community Pushes Ethereum Founder To The Brink # Cryptocurrency # BTC # Ethereum http://www.hackingcrypto.com/posts/I9SQP/crypto-community-ethereum-founder-vitalik-buterin-cryptocurrency-warning …
Ripple Price Increases, Beats Ethereum To Come Below Bitcoin # xrp # Ripple # cryptocurrency # ethereum http://krl.io/xrp
Bitcoin Alternatives: Ethereum Vs Litecoin Vs Verge Vs Ripple Vs Zcash https://www.ndtv.com/business/bitcoin-alternatives-ethereum-vs-litecoin-vs-verge-vs-ripple-vs-zcash-1794177 … via @ ndtv
This is an example 

Where will you be when Mavro launches...2018 appraoches quickly! # cryptocurrency # blockchain # crypto # bitcoin # bitcoins # bitcointrading # blockchain # crypto # cointelegraph # bitcoinprice # ico # ethereum # litecoin # eth # ltc # bcc # bitcoincash # mavropic.twitter.com/0dFNDXBO2d
Bitcoin Cash: $2,380 +8.92% (+$194.98) High: $2,470 Low: $2,135.00 Volume: 3046 $ BCC # BCC # bitcoincash
$ BCC # BCC # bitcoincash tango down by # xrb $ xrb # raiblocks 19th place and still growing :) $ btc $ xrp $ eth $ bch $ ada $ ltc $ iota $ xem $ dash $ xlm $ xmr $ eos $ neo $ qtum $ trx $ etc $ xvg $ lsk
Happy New Year Crypto Folks # Bitcoin # BitcoinCash # BTC # BTG # BCD # ethereum # ETH # EtherDelta # ICO # cryptocurrency # CryptoCurrencies # Ripple # XRP # usdt # BCC # BCH # Bankera # Play2Live # FinTech # Trueflip # TravelChainpic.twitter.com/wUDJDCwONf
Bitcoin Cash / BCC $ BCH crossed 10-day low (0.17240000 BTC, 23.15% change). # turtlessellhttps://twitter.com/GetCoinAlerts/status/94320230

augto, LTC LITECOIN EXCHANGE - NO ID VERIFICATION ➜➜ …http://binance.cryptocurrency.trading.gewgool.com/b/glosmwezf523945007 … XRP ♳ IOTA ♰ LTC ♬ BCC ☻ XVG BUY ☆ ETH 523945007kgidjzqyc tyuogjzfd
This is an example on someone blinded by money, who spins any type of narrative led by manipulation to fulfill any sense of morality in his small brain. This isn't a scaling issue, it's control. Disclaimer: I own Bitcoin, Bitcoin Cash, Ethereum, Litecoin, Monero, EOS, and Qtum.
Crypto makes my blood pressure go bullish. # Crypto # blockchain # Bitcoin # litecoin # Ethereum # Dash # Ripple $ BTC $ LTC $ ETH $ DASH $ XRP $ XVG
Yo @ 6BillionPeople ! you in on @ ElectracoinECA??? Get in before its too late bro. Im 20X right now! # CryptoCurrencies # bitcoin # Litecoin # ETHEREUM # Ripple
LTC to GBP price £172.52 https://goo.gl/2k5EnC # litecoin # ltc # cryptocurrency
LTC to BTC price B0.016653 https://goo.gl/XdxCYk # litecoin # ltc # cryptocurrency
LTC to USD price $233.16 https://goo.gl/e4D8WX # l

[Eos] カルカッタの同盟先がヴェネツィアからイングランドに変わったようです。 https://search.meixiaojie.com/worldinfo/eos/75/ … (01/01 08:55)
[Eos] カルカッタの同盟先がヴェネツィアからイングランドに変わったようです。 https://search.meixiaojie.com/worldinfo/eos/75/ … (01/01 08:55)
미오 : 가스트와 뮤지컬이 슈퍼에 갔는데, 뮤지컬이 구미가 먹고싶은지 바구니에 담자 가스트가 구미를 도로 뺐어. 근데 뮤지컬이 시무룩해하자 가스트는 뭔가 가져오라고 시켰고, 뮤지컬이 다른데로 간 사이 몰래 구미를 바구니에 숨기듯 넣더라. 좋을때야~♪
泊まるホテルは東横インばかりなのでスーパーホテルはわからないですねー。 東横インを選択する理由は安いからですねw
Value of $ EOS is: $8.79 Kucoin has over 200,000 users registered! Join us today! http://kucoin.com/#/?r=E3aQNa $ TFL $ COVAL $ PRG $ PPT $ STEEM $ VRM $ ATMS $ QAU $ OBITS $ VTR $ PTOY $ XEM $ SONM $ ARGS $ CURE $ SAN $ KORE $ CNX $ WABI $ INFX $ MEME $ BCPT $ RISEpic.twitter.com/QoxG8JwxLY
Godt nytt år! [Bilder: Canons EOS 70D] pic.twitter.com/asZMHOrJ8T
そうだったんですか？ そもそも武井先生、三刊王ならぬミカン王の先生なんですよね…
【デジタル一眼 売れ筋ランキング 第8位】 Canon デジタル一眼レフカメラ EOS 9000D レンズキット EF-S18-135mm F, http://amzn.to/2CuXj2n
以前に冬コミ遠征でこのホテルに泊まった時は元日に年賀状を出したりしましたし結構サービスは良いですねぇ 朝食にご当地メニュー出ますし – bei スーパーホテル
"Consagre ao Senh

,id,symbol,coin_name,coin_price,date,compound_average,positive_average,neutral_average,negative_average
BTC,1182,BTC,bitcoin,13444.9,2018-01-01,0.140736,0.08964,0.87804,0.0322
ETH,7605,ETH,ethereum,756.2,2018-01-01,0.23534,0.10588,0.88388,0.01024
XRP,5031,XRP,ripple,2.05,2018-01-01,0.417144,0.20968,0.77628,0.01408
BCH,202330,BCH,bitcoin cash / bcc,2319.12,2018-01-01,0.08062,0.05456,0.89392,0.05152
ADA,321992,ADA,cardano,0.7022,2018-01-01,0.215136,0.10432,0.87684,0.0188


In [45]:
file_name = os.path.join("resources", "coin_tweet_analysis_7_days_20180101.csv")
coin_data_df.to_csv(file_name)
